In [7]:
import json
import pennylane as qml
import pennylane.numpy as np

dev = qml.device('default.qubit', wires = 2)

@qml.qnode(dev)
def trotterize(alpha, beta, time, depth):
    # Put your code here #
    for i in range(depth):
        qml.IsingXX(alpha*2*(time/depth), wires=[0,1])
        qml.IsingZZ(beta*2*(time/depth), wires=[0,1])

    return qml.probs()
    # Return the probabilities

# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    dev = qml.device("default.qubit", wires=2)
    ins = json.loads(test_case_input)
    output = list(trotterize(*ins).numpy())

    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-4
    ), "Your circuit does not give the correct probabilities."

    tape = trotterize.qtape

    names = [op.name for op in tape.operations]
    
    assert names.count('ApproxTimeEvolution') == 0, "Your circuit is using the built-in PennyLane Trotterization!"
    assert names.count('Evolve') == 0, "Your circuit is using the built-in PennyLane Trotterization!"
    assert names.count('QubitUnitary') == 0, "Can't use custom-built gates!"

# These are the public test cases
test_cases = [
    ('[0.5,0.8,0.2,1]', '[0.99003329, 0, 0, 0.00996671]'),
    ('[0.9,1.0,0.4,2]', '[0.87590286, 0, 0, 0.12409714]')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)
        print (output)
    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[0.5,0.8,0.2,1]'...
[0.9900332889206209, 0.0, 0.0, 0.009966711079379187]
Correct!
Running test case 1 with input '[0.9,1.0,0.4,2]'...
[0.8759028645704475, 0.0, 0.0, 0.12409713542955253]
Correct!
